In [1]:
import ee
ee.Initialize(project='pyregence-ee')
import datetime
import scripts.analysis_functions as af
import scripts.utils as utils
%load_ext autoreload
%autoreload 2

### Run BS Mapper for NIFC 2021 and 2022 fire sets

In [ ]:
# One set of yearly fires, no need for remove_recent() or year/ac filtering, that was already done upon data fetch
# NIFC
yr='2022'
fires_asset = f"projects/pyregence-ee/assets/conus/nifc/nifc_fires_{yr}_gte100ac_20230406"
fires = ee.FeatureCollection(fires_asset)
print(f'Total Fires in FeatureCollection: {fires.size().getInfo()}')

# filter 2022 fires for a July 1 cut-off (was for FireFactor V2)
# if yr=='2022':
#     fires = fires.filter(ee.Filter.lte('Discovery','2022-07-01'))
#     print(fires.sort('Discovery',False).first().getInfo()['properties'])
#     print(f'Fires after filter: {fires.size().getInfo()}')

# return BS ee.Image for each fire ee.Feature
bs_imgColl = ee.FeatureCollection(fires).map(af.bs_calc_new)

# composite that ee.ImageCollection with a max() reducer
annual_conus_bs = ee.ImageCollection(bs_imgColl).max().add(1).rename('SEVERITY').set('Year',int(yr)) #shift severity values +1 so 0 can be nodata
# To Asset
desc = f'nifc_bs_{yr}_gte100acres_20230406'
utils.exportImgtoAsset(annual_conus_bs, 
                    desc=desc,
                    region=None,
                    asset_folder= "projects/pyregence-ee/assets/conus/nifc", 
                    export_type='conus',
                    export=True)
                    #default=sensor)
       

In [ ]:
# Running for MTBS 2020
fires = ee.FeatureCollection("projects/pyregence-ee/assets/conus/mtbs/mtbs_fires_2020_20230406")
print(f'Total Fires in FeatureCollection: {fires.size().getInfo()}')

bs_imgColl = ee.FeatureCollection(fires).map(af.bs_calc_new)
annual_conus_bs = ee.ImageCollection(bs_imgColl).max().add(1).rename('SEVERITY').set('Year',2020) #shift severity values +1 so 0 can be nodata

# To Asset
desc = f'mtbs_bs_2020_20230406'
utils.exportImgtoAsset(annual_conus_bs, 
                    desc=desc,
                    region=None,
                    asset_folder= "projects/pyregence-ee/assets/conus/mtbs", 
                    export_type='conus',
                    export=True)
                    #default=sensor)
print('\n')

## Run BS Mapper for MTBS yearly fire perims in a for-loop

In [ ]:
# Running for each year of MTBS fire perims 
fires = ee.FeatureCollection("projects/pyregence-ee/assets/conus/mtbs/mtbs_fires_all_20220928")
print(f'Total Fires in FeatureCollection: {fires.size().getInfo()}')
# acre_min = 0 # can filter by Acres field 
years = [2013,2014,2015,2016,2017,2018,2019,2020]
# print(fires.sort('Year',False).first().getInfo()['properties'])
# fires = fires.filter(ee.Filter.eq('Year',2021))
# print(fires.size().getInfo())
for y in years:
    fires_f = fires.filter(ee.Filter.eq('Year',int(y)))
    # fires_w_filters = gic.remove_recent(fires).filter(ee.Filter.And(ee.Filter.gte('BurnBndAc', acre_min),ee.Filter.eq('Year',y)) )
    print(f'Computing Burn Severity for {fires_f.size().getInfo()} fires in {y}: ')

    # latest_fire = fires_w_filters.sort('Discovery',False).first()
    # print('latest fire: ', latest_fire.get('Discovery').getInfo())

    # run_mode = fi.config_mode(latest_fire)
    # print(f'run mode: {run_mode}')
    
    bs_imgColl = ee.FeatureCollection(fires_f).map(af.bs_calc_new)
    annual_conus_bs = ee.ImageCollection(bs_imgColl).max().add(1).rename('SEVERITY').set('Year',y) #shift severity values +1 so 0 can be nodata
    
    # To Asset
    desc = f'mtbs_annual_bs_{str(y)}_ImproveWindowing'
    utils.exportImgtoAsset(annual_conus_bs, 
                        desc=desc,
                        region=None,
                        asset_folder= "projects/pyregence-ee/assets/conus/mtbs", 
                        export_type='conus',
                        export=False)
                        #default=sensor)
    print('\n')

## MTBS annual fire perims 2013-2020

# For one fire (takes a Feature)
### Select a fire from a pre-existing fire featurecollection or provide your own fire feature asset (must have 'Discovery' property with value of format ee.String('yyyy-mm-dd') )


## Exports
### To export an annual set of fires, use export_type = 'conus', if its a single fire, use export_type = 'single_fire'

In [ ]:
CA = ee.FeatureCollection("TIGER/2018/States").filter(ee.Filter.eq('NAME','California')).first()
# Map.addLayer(ee.Feature(CA))
# print(ee.Feature(CA).getInfo()['properties'])
                                                      
# Single Fire export templates
# # To Google Drive
# desc = 'dixie_bs2020_test_20211227'
# utils.exportImgtoDrive(miller,
#                         desc,
#                         export_type='single_fire',
#                         folder='BurnSeverity_outputs',
#                         export=True,
#                         default=sensor)


# # To Asset
# desc = 'dixie_bs2020_test_20211227'
# utils.exportImgtoAsset(miller, 
#                         desc=desc, 
#                         asset_folder= "users/kwoodward/burn-severity/bs_images", 
#                         export_type='single_fire',
#                         export=True,
#                         default=sensor)

# # CONUS/multi-fire export template
# # To Google Drive
# desc = 'CA_bs2021_nifcFires_20220713'
# utils.exportImgtoDrive(annual_conus_bs.clip(CA),
#                         desc,
#                         export_type='conus',
#                         folder='BurnSeverity_outputs',
#                         export=False)
#                         #default=sensor)


# To Asset
desc = 'CA_bs2021_nifcFires_20220713'
utils.exportImgtoAsset(annual_conus_bs.clip(CA), 
                        desc=desc, 
                        asset_folder= "users/kwoodward/burn-severity/bs_images", 
                        export_type='conus',
                        export=True)
                        #default=sensor)

## Running a single fire

In [2]:
# Running for a particular fire

# Select the fire based on irwinID (or name and edit filter below)
#fire_name = 'KNP COMPLEX' 
#fire_irwin = '499D6545-2A19-458A-8C7C-554932C6B86E'
#fire_name = 'WINDY' #there are 2 WINDY fires, using irwinID # fire_irwin = 'ACD5C207-BB83-4419-977D-3F64778081A3'
fire_irwin = 'ACD5C207-BB83-4419-977D-3F64778081A3'

# Read in fire data
fires = ee.FeatureCollection('projects/pyregence-ee/assets/conus/mtbs/mtbs_fires_2021_20230810')
print(f'Total Fires in FeatureCollection: {fires.size().getInfo()}')
#print(fires.first().getInfo()['properties'])
#print(fires.aggregate_array('Incid_Name').distinct().sort().getInfo())

# Filter to desired fire
#fires_f = fires.filter(ee.Filter.eq('Incid_Name',fire_name))
fires_f = fires.filter(ee.Filter.eq('irwinID',fire_irwin))
print(f'Filtered fire size: {fires_f.size().getInfo()}')

fire_name = fires_f.first().get('Incid_Name').getInfo().lower() #brittle, make sure only 1 feature
print(fire_name)

# Run burn severity tool 
bs_coll = ee.FeatureCollection(fires_f).map(af.bs_calc_new)
#bs_fire = af.bs_calc_new(fires_f.first()) #testing direct function call
#print(bs_fire.propertyNames().getInfo())
# while max() isn't really needed for 1 fire, keeping the pattern of the multi-fire runs
bs_composite = ee.ImageCollection(bs_coll).max().add(1).rename('SEVERITY') #shift severity values +1 so 0 can be nodata

# To Asset
# get current date to put into asset name
cur_date = datetime.datetime.now().strftime("%Y%m%d")
desc = f'mtbs_bs_{str(fire_name).replace(" ", "")}_{str(cur_date)}'
print(desc)
#fails with Image.clipToBoundsAndScale: The geometry for image clipping must be bounded.
utils.exportImgtoAsset(bs_composite,  
                    desc=desc,
                    region=bs_coll.geometry(),
                    asset_folder='projects/pyregence-ee/assets/fires_bs_tool/dmn_testing', 
                    export_type='single_fire', 
                    export=False) 


Total Fires in FeatureCollection: 1022
Filtered fire size: 1
windy


## ee authentication block

In [ ]:
# import ee
# ee.Authenticate()